# Momentum correction and alignment of already binned data
This example shows how to use the momentum correction and alignment features of `sed-processor` with data that come already in a binned format - e.g. a tiff file stored from a momentum microscope. For demonstration purposes, we will use the WSe2 data from [tutorial 2](2_conversion_pipeline_for_example_time-resolved_ARPES_data.ipynb), and bin these data into the raw coordinates X/Y/TOF, similar as they would come from a momentum microscope tiff file.

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import sed
from sed.dataset import dataset

%matplotlib widget

## Load Data

In [ ]:
dataset.get("WSe2") # Put in Path as root_dir="" to a storage of at least 20 GByte free space, otherwise local folder will be used.
data_path = dataset.dir # This is the path to the data
scandir, caldir = dataset.subdirs # scandir contains the data, caldir contains the calibration files

In [ ]:
# create sed processor using the config file:
sp = sed.SedProcessor(folder=scandir, config="../src/sed/config/mpes_example_config.yaml", user_config={}, system_config={}, verbose=True)
sp.add_jitter()

Generate binned dataset

In [ ]:
binned_data = sp.pre_binning()

## Distortion correction and alignment for binned data
### 1. step: load data into the momentum corrector class instance
If data comes as xarray with defined axes ranges, it can just be passed directly. If the data are just a plain numpy array, also provide the detector coordinate ranges of the binned data. The interactive tool allows you to select a slice for correction.

In [ ]:
# sp.mc.load_data(binned_data) # as xarray DataArray
bin_ranges = ((binned_data.X[0], binned_data.X[-1]), (binned_data.Y[0], binned_data.Y[-1]), (binned_data.t[0], binned_data.t[-1]))
sp.mc.load_data(binned_data.data, bin_ranges=bin_ranges) # as np.array
sp.mc.select_slicer(plane=33, width=10, apply=True)

### 2. Step:
Next, we select a number of features corresponding to the rotational symmetry of the material, plus the center. These can either be auto-detected (for well-isolated points), or provided as a list (these can be read-off the graph in the cell above).
These are then symmetrized according to the rotational symmetry, and a spline-warping correction for the x/y coordinates is calculated, which corrects for any geometric distortions from the perfect n-fold rotational symmetry.

In [ ]:
#features = np.array([[203.2, 341.96], [299.16, 345.32], [350.25, 243.70], [304.38, 149.88], [199.52, 152.48], [154.28, 242.27], [248.29, 248.62]])
#sp.mc.define_features(features=features, rotation_symmetry=6, include_center=True, apply=True)
# Manual selection: Use a GUI tool to select peaks:
#sp.mc.feature_select(rotation_symmetry=6, include_center=True)
# Autodetect: Uses the DAOStarFinder routine to locate maxima.
# Parameters are:
#   fwhm: Full-width at half maximum of peaks.
#   sigma: Number of standard deviations above the mean value of the image peaks must have.
#   sigma_radius: number of standard deviations around a peak that peaks are fitted
sp.define_features(rotation_symmetry=6, auto_detect=True, include_center=True, fwhm=10, sigma=12, sigma_radius=4, apply=True)

### 3. Step: 
Generate nonlinear correction using the splinewarp algorithm. If no landmarks have been defined in the previous step, default parameters from the config are used

In [ ]:
# Option whether a central point shall be fixed in the determination fo the correction
sp.mc.spline_warp_estimate(include_center=True)

In [ ]:
sp.mc.pose_adjustment(xtrans=8, ytrans=7, angle=-4, apply=True)

### 4. Step: Apply correction to the binned dataset and visualize the results
This function uses interpolation to apply the generated displacement field to the whole binned dataset. There might be some loss of data quality due to this process.
Also, make difference data between original/binned and corrected data for direct visualisation.

In [ ]:
corrected_data = binned_data.copy()
corrected_data.data = sp.mc.apply_correction(binned_data.data, axis=2)
difference_data = corrected_data - binned_data

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,3), layout='constrained')
binned_data.isel(t=40).T.plot(ax=ax[0], cmap="terrain_r")
ax[0].set_title('corrected data')
difference_data.isel(t=40).T.plot(ax=ax[1])
ax[1].set_title('difference data')